## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-12-17-47-23 +0000,wapo,Afghanistan and Pakistan claim to kill dozens ...,https://www.washingtonpost.com/world/2025/10/1...
1,2025-10-12-17-45-22 +0000,nyt,Israelis and Palestinians Await Hostage-Prison...,https://www.nytimes.com/2025/10/12/world/middl...
2,2025-10-12-17-43-27 +0000,bbc,China accuses US of 'double standards' over ta...,https://www.bbc.com/news/articles/cn828kg8rmzo...
3,2025-10-12-17-31-50 +0000,nypost,Father of Israeli-American hostage assumed mur...,https://nypost.com/2025/10/12/world-news/hosta...
4,2025-10-12-17-23-38 +0000,nyt,Cars to Fighter Jets: China’s New Export Curbs...,https://www.nytimes.com/2025/10/12/business/ch...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2372/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
46,trump,28
67,gaza,11
76,shutdown,10
126,killed,10
152,dead,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
57,2025-10-12-12-36-07 +0000,nypost,Hamas ready to begin freeing 20 living Israeli...,https://nypost.com/2025/10/12/world-news/hamas...,76
181,2025-10-11-20-36-00 +0000,wsj,Trump said funds have been identified to pay m...,https://www.wsj.com/politics/policy/trump-says...,73
201,2025-10-11-18-12-00 +0000,nypost,President Trump will make sure military troops...,https://nypost.com/2025/10/11/us-news/presiden...,69
120,2025-10-12-04-37-28 +0000,bbc,Israelis praise Trump at huge rally ahead of e...,https://www.bbc.com/news/articles/c75q5qng0rgo...,67
153,2025-10-11-23-27-30 +0000,nyt,Trump Says He Will Pay Troops Despite Governme...,https://www.nytimes.com/2025/10/11/us/politics...,64


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
57,76,2025-10-12-12-36-07 +0000,nypost,Hamas ready to begin freeing 20 living Israeli...,https://nypost.com/2025/10/12/world-news/hamas...
42,44,2025-10-12-14-45-04 +0000,nypost,"At least 4 killed, 20 injured in mass shooting...",https://nypost.com/2025/10/12/us-news/4-dead-a...
201,41,2025-10-11-18-12-00 +0000,nypost,President Trump will make sure military troops...,https://nypost.com/2025/10/11/us-news/presiden...
200,40,2025-10-11-18-19-42 +0000,cbc,"Tennessee authorities confirm 16 dead, no surv...",https://www.cbc.ca/news/world/tennessee-explos...
171,31,2025-10-11-21-29-00 +0000,wsj,Budget chief Russell Vought had vowed to fire ...,https://www.wsj.com/politics/policy/what-to-kn...
111,29,2025-10-12-07-22-45 +0000,nypost,"Chinese coast guard fires water cannon, rams P...",https://nypost.com/2025/10/12/world-news/chine...
0,26,2025-10-12-17-47-23 +0000,wapo,Afghanistan and Pakistan claim to kill dozens ...,https://www.washingtonpost.com/world/2025/10/1...
120,26,2025-10-12-04-37-28 +0000,bbc,Israelis praise Trump at huge rally ahead of e...,https://www.bbc.com/news/articles/c75q5qng0rgo...
52,24,2025-10-12-13-09-30 +0000,nypost,Trump announces Dan Scavino as new head of Whi...,https://nypost.com/2025/10/12/us-news/trump-an...
36,23,2025-10-12-15-23-44 +0000,cbc,Canada's tariffs on Chinese EVs created '2-fro...,https://www.cbc.ca/news/canada/manitoba/kinew-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
